In [ ]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/12F9epgBqkVCww8VcBeD2S2CIfaLNe0zC?usp=share_link"
gdown.download_folder(url, quiet=True)

In [ ]:
!pip install tqdm
!pip install dlib

In [ ]:
# number of aligned images:
# part_1: 18.358/20.274
# part_2: 21.110/21.438
# part_3: 6.356/6.500

import os
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import shutil
import zipfile
import dlib
from tqdm import tqdm
from google.colab.patches import cv2_imshow

shape_predictor_path = "/content/biometric_systems_project/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector() # HOG-based detector
predictor = dlib.shape_predictor(shape_predictor_path)

def shape_to_normal(shape):
  shape_normal = []
  # shape[0]:shape[16] = jaw line
  # shape[17]:shape[21] = left eyebrow
  # shape[22]:shape[26] = right eyebrow
  # shape[27]:shape[35] = nose (27-30 nose bridge + 31-35 lower nose)
  # shape[36]:shape[41] = left eye
  # shape[42]:shape[47] = right eye
  # shape[48]:shape[59] = outer lip
  # shape[60]:shape[68] = inner lip
  for i in range(0, shape.num_parts):
    shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
  return shape_normal

def get_eyes_nose_dlib(shape):
  nose = shape[30][1]
  left_eye_x = (shape[36][1][0] + shape[39][1][0]) // 2
  left_eye_y = (shape[36][1][1] + shape[39][1][1]) // 2
  right_eyes_x = (shape[42][1][0] + shape[45][1][0]) // 2
  right_eyes_y = (shape[42][1][1] + shape[45][1][1]) // 2

  return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

def get_eyes_in_image(image, top, bottom, left, right, left_eye, height_face):
  left_eye_y = left_eye[1]
  while left_eye_y > bottom:
    bottom += int(height_face * 0.02)
  image = image[top:bottom, left:right]
  return image

def get_nose_in_image(image, top, bottom, left, right):
  image = image[top:bottom, left:right]
  return image

def get_mouth_in_image(image, top, bottom, left, right):
  image = image[top:bottom, left:right]
  return image

def align_face(image, file_name):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = detector(gray)
  if len(faces) > 0:
    for face in faces:
    # Extract face bounding box coordinates
      shape = predictor(gray, face)

      # Get facial landmarks for the detected face
      shape = shape_to_normal(shape)

      #for _, (lx, ly) in shape:
      #  cv2.circle(image, (lx, ly), radius=3, color=(255, 0, 0), thickness=-1)  # Blue dots

      # Get the nose, left eye, and right eye positions
      nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
      #cv2.circle(image, nose, radius=5, color=(0, 255, 0), thickness=-1)  # Green dot for the nose
      #cv2.circle(image, left_eye, radius=5, color=(0, 0, 255), thickness=-1)  # Red dot for left eye
      #cv2.circle(image, right_eye, radius=5, color=(0, 0, 255), thickness=-1)  # Red dot for right eye

      delta_x = right_eye[0] - left_eye[0]
      delta_y = right_eye[1] - left_eye[1]
      # angle from radians to degrees
      angle = math.atan2(delta_y, delta_x) * (180 / math.pi)

      center_of_eyes = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
      rotation_matrix = cv2.getRotationMatrix2D(center_of_eyes, angle, scale=1.0)

      h, w = image.shape[:2]

      aligned_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
      #cv2.rectangle(aligned_image, (face.left(), face.top()), (face.right(), face.bottom()), (255, 255, 255), 2)

      # catch features in the image
      height_face = face.bottom() - face.top()
      eyes_bottom = face.top() + int(height_face * 0.4)
      #nose_bottom = eyes_bottom +int(height_face * 0.2)
      left = max(0, face.left())
      right = min(aligned_image.shape[1], face.right())
      eyes_image = get_eyes_in_image(aligned_image, face.top(), eyes_bottom, left, right, left_eye, height_face)
      nose_top = shape[28][1][1]
      nose_bottom = shape[33][1][1] + 3
      nose_image = get_nose_in_image(aligned_image, nose_top, nose_bottom, left, right)
      mouth_image = get_mouth_in_image(aligned_image, nose_bottom, face.bottom(), left, right)

      # save the catched features into image
      os.makedirs("eyes", exist_ok=True)
      os.makedirs("nose", exist_ok=True)
      os.makedirs("mouth", exist_ok=True)
      if eyes_image.size == 0:
        continue
      if nose_image.size == 0:
        continue
      if mouth_image.size == 0:
        continue
      cv2.imwrite(os.path.join("eyes", "eyes_" + file_name), eyes_image)
      cv2.imwrite(os.path.join("nose", "nose_" + file_name), nose_image)
      cv2.imwrite(os.path.join("mouth", "mouth_" + file_name), mouth_image)

      # boolean=True, image is aligned
      return True
  return False

def main():
  folder_path = "/content/biometric_systems_project"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  aligned_faces, total_faces = 0, 0
  dataset_path = folder_path + "/dataset_part_3/"
  for file_name in tqdm(os.listdir(dataset_path), desc="Processing Images"):
    if not file_name.endswith(".jpg"):
      continue
    total_faces += 1
    if file_name.startswith("."):
      file_name = file_name[2:]
    image = cv2.imread(os.path.join(dataset_path, file_name))
    boolean = align_face(image, file_name)
    if boolean:
      aligned_faces += 1
  print(f"Total faces: {total_faces}")
  print(f"Aligned faces: {aligned_faces}")
  shutil.make_archive("eyes", "zip", "eyes")
  shutil.make_archive("nose", "zip", "nose")
  shutil.make_archive("mouth", "zip", "mouth")
if __name__ == "__main__":
  main()
